In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from splits import split_users # contains split_users func

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 100)

In [4]:
# # read in pickled DFs generated by query_dfs.py

# df_orders = pd.read_pickle("./pickle/df_orders.pickle")
# df_train = pd.read_pickle("./pickle/df_train.pickle")
# df_prior = pd.read_pickle("./pickle/df_prior.pickle")
# df_prod_detail = pd.read_pickle("./pickle/df_prod_detail.pickle")

In [5]:
# read in pickled feature DF generated by feature_engineering_1.ipynb
X = pd.read_pickle("./pickle/X_F.pickle")

In [ ]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

In [ ]:
# X_train, X_test, y_train, y_test = split_users(X, test_size=0.2)# only using 10%


In [ ]:


# rf = RandomForestClassifier(n_estimators=500, max_depth=8, max_features=8, n_jobs=-1)
# rf_fit = rf.fit(X_train, y_train)


In [ ]:
# preds = rf_fit.predict(X_test)

In [ ]:
# f1_score(y_test, preds)

In [ ]:
# with open(f"models/rf_fit.pickle", "wb") as pfile:
#         pickle.dump(rf_fit, pfile)

## Grid Search

In [ ]:
X_train, X_test, y_train, y_test = split_users(X, subset=0.02, test_size=0.2)

In [ ]:
# estimator = XGBClassifier(objective='binary:logistic',
#                           use_label_encoder=False,
#                           eval_metric='logloss',
#                           random_state=54,
#                           learning_rate=0.01,
# )

# params = {
#     'max_depth': [7, 8, 9],
#     'n_estimators': [400, 500],
#     'colsample_bytree': [0.6, 0.7, 0.8],
#     'min_child_weight': [7, 8, 9]}

In [ ]:
# grid_search = GridSearchCV(
#     estimator = estimator,
#     param_grid = params,
#     verbose=10
# )

In [ ]:
# %%time
# grid_xgb = grid_search.fit(X_train, y_train)
# print("The best parameters are: \n", grid_search.best_params_)



In [ ]:
# with open(f"models/grid_xgb.pickle", "wb") as pfile:
#         pickle.dump(grid_xgb, pfile)

## Grid Search Results

**On a different VM, I used grid search to tune min_child_weight and colsample_bytree parameters. Here was the grid & results:**

```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=54,
                          max_depth=3,
                          learning_rate=0.01,
                          n_estimators=500
)

params = {
    'min_child_weight': range(1, 10, 1),
    'colsample_bytree': [.6, .7, .8, .9, 1.0]
}

# results
Fitting 5 folds for each of 45 candidates, totalling 225 fits
The best parameters are: 
 {'colsample_bytree': 0.7, 'min_child_weight': 8}
CPU times: user 1h 47min 29s, sys: 1.55 s, total: 1h 47min 30s
Wall time: 10min 54s

```

**Below were the parameters for our grid search.**
```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=54,
                          learning_rate=0.01,
)

params = {
    'max_depth': [7, 8, 9],
    'n_estimators': [400, 500],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'min_child_weight': [7, 8, 9]}
```

**And the results:**

```
Best results are:
{'colsample_bytree': 0.8, 'max_depth': 7, 'min_child_weight': 9, 'n_estimators': 400}
CPU times: user 21h 33min 24s, sys: 18.2 s, total: 21h 33min 42s
```

Great. Now we have our parameters. Let's run the model!


In [6]:
# X_train, X_test, y_train, y_test = split_users(X, subset=False, test_size=0.2)


    X_train sample size: 6782401
    X_test sample size: 1692260


In [ ]:
# %%time
# xgb = XGBClassifier(colsample_bytree=0.8,
#                     min_child_weight=9,
#                     n_estimators=400,
#                     max_depth=7,
#                     learning_rate=0.009,
#                     eval_metric='logloss',
#                     verbosity=3,
#                     use_label_encoder =False)

# xgb_fit = xgb.fit(X_train, y_train)

In [7]:
# with open(f"models/xgboost_fit_all.pickle", "wb") as pfile:
#         pickle.dump(xgb_fit, pfile)

NameError: name 'xgb_fit' is not defined

In [ ]:
# preds_all = xgb_fit.predict(X_test)

In [ ]:
# xgb_all_score = f1_score(y_test, preds_all)
# xgb_all_score

Now let's try a couple more, each time without one of the following features that may/may not be helpful
* log features
* streak
* streak_abs

In [8]:
X.columns

Index(['product_id', 'user_id', 'user_total_prod_orders', 'cart', 'in_cart',
       'last_cart', 'in_last_cart', 'prod_prior_sales', 'prod_pct_reorders',
       'prod_avg_atco', 'user_avg_cart_size', 'days_since_prior_order',
       'order_hour_of_day', 'user_avg_spacing', 'streak', 'streak_nan',
       'up_buy_streak', 'up_n5_n_buys', 'up_n5_buy_ratio', 'up_atco_sum',
       'up_atco_avg', 'prod_total_mkt_share', 'prod_total_mkt_share_log',
       'aisle_total_sales', 'prod_aisle_mkt_share', 'prod_aisle_mkt_share_log',
       'dpt_total_sales', 'prod_dpt_mkt_share', 'prod_dpt_mkt_share_log',
       'streak_abs'],
      dtype='object')

In [9]:
X_2 = X.drop(columns=["prod_dpt_mkt_share_log", "prod_aisle_mkt_share_log"])

In [10]:
X_train, X_test, y_train, y_test = split_users(X_2, subset=False, test_size=0.2)


    X_train sample size: 6782401
    X_test sample size: 1692260


In [ ]:
xgb_no_log = XGBClassifier(colsample_bytree=0.8,
                               min_child_weight=9,
                               n_estimators=400,
                               max_depth=7,
                               learning_rate=0.009,
                               eval_metric='logloss',
                               verbosity=3,
                               use_label_encoder =False)




[02:44:46] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:146: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[02:44:46] DEBUG: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:154: Using tree method: 1
[02:44:46] DEBUG: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:119: Using updaters: grow_histmaker,prune
[02:44:46] ======== Monitor: TreePruner ========


KeyboardInterrupt: 

[02:44:55] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:44:59] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:03] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7


KeyboardInterrupt: 

[02:45:10] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:14] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:17] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 250 extra nodes, 0 pruned nodes, max_depth=7
[02:45:21] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:25] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7


KeyboardInterrupt: 

[02:45:33] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 246 extra nodes, 0 pruned nodes, max_depth=7
[02:45:37] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:40] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:44] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:48] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:45:52] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

KeyboardInterrupt: 

[02:46:00] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:46:04] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 246 extra nodes, 0 pruned nodes, max_depth=7
[02:46:07] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:46:11] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:46:15] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:46:18] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[02:48:57] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:49:00] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 252 extra nodes, 0 pruned nodes, max_depth=7
[02:49:04] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:49:08] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7
[02:49:12] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 254 extra nodes, 0 pruned nodes, max_depth=7


In [ ]:
xgb_fit_no_log = xgb_no_log.fit(X_train, y_train)

In [ ]:
xgb_fit_no_log

In [ ]:
preds_no_log = xgb_fit_no_log.predict(X_test)

In [ ]:
xgb_no_log_score = f1_score(y_test, preds_no_log)
xgb_no_log_score

In [ ]:
with open(f"models/xgboost_fit_no_log.pickle", "wb") as pfile:
        pickle.dump(xgb_fit_no_log, pfile)

In [ ]:
X_3 = X.drop(columns="streak")
X_train, X_test, y_train, y_test = split_users(X_3, subset=False, test_size=0.2)

xgb_no_streak = XGBClassifier(colsample_bytree=0.8,
                               min_child_weight=9,
                               n_estimators=400,
                               max_depth=7,
                               learning_rate=0.009,
                               eval_metric='logloss',
                               verbosity=3,
                               use_label_encoder =False)

xgb_fit_no_streak = xgb_no_streak.fit(X_train, y_train)


In [ ]:
preds_no_streak = xgb_fit_no_streak.predict(X_test)


In [ ]:

xgb_score_no_streak = f1_score(y_test, preds_no_streak)
xgb_score_no_streak

In [ ]:
with open(f"models/xgboost_fit_no_streak.pickle", "wb") as pfile:
        pickle.dump(xgb_fit_no_streak, pfile)

In [ ]:
X_4 = X.drop(columns="streak_abs")
X_train, X_test, y_train, y_test = split_users(X_4, subset=False, test_size=0.2)

xgb_no_streak_abs = XGBClassifier(colsample_bytree=0.8,
                               min_child_weight=9,
                               n_estimators=400,
                               max_depth=7,
                               learning_rate=0.009,
                               eval_metric='logloss',
                               verbosity=3,
                               use_label_encoder =False)

xgb_fit_no_streak_abs = xgb_no_streak_abs.fit(X_train, y_train)
preds_no_streak_abs = xgb_fit_no_streak_abs.predict(X_test)

xgb_score_no_streak_abs = f1_score(y_test, preds_all)
xgb_score_no_streak_abs

In [ ]:
with open(f"models/xgboost_fit_no_streak_abs.pickle", "wb") as pfile:
        pickle.dump(xgb_fit_no_streak_abs, pfile)

And now we'll do the same without `colsample_bytree` and `min_child_weight`. We'll call this model `xgb_fit_2`.

We'll then compare results.